# Instalacion de librerias adicionales

In [1]:
! pip install opendatasets --user

# Importar Librerias

In [2]:
import opendatasets as od
import pandas as pd
import requests

# Descargar el dataset

In [3]:
dataset_url='https://www.kaggle.com/datasets/juanmartnrodrguez/argentina-dataset-propiedades'
od.download(dataset_url)

100%|██████████| 255M/255M [00:39<00:00, 6.69MB/s] 


In [4]:
df = pd.read_csv('./argentina-dataset-propiedades/ar_properties.csv')
df

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,wdQ5hWhv8P14T7Sh9g4QCg==,Propiedad,2020-12-25,9999-12-31,2020-12-25,-32.716652,-68.642692,Argentina,Mendoza,NaN,...,NaN,350.0,350.0,NaN,NaN,NaN,Excelentes Lotes Sobre Ruta 34,Corredor Responsable: VICTOR E. MONTIVERO - C....,Lote,Venta
1,nnMBYZ4RMRY+vm753EtA+g==,Propiedad,2020-12-25,9999-12-31,2020-12-25,-24.797723,-65.467514,Argentina,Salta,NaN,...,NaN,1541.0,1541.0,NaN,NaN,Mensual,TERRENO + VENTA + JARDINES DE SAN LORENZO +150...,Corredor Responsable: Pablo Castañeda - C.U.C....,Lote,Venta
2,+dnVA1K6JxzL1zAjOEQ1pA==,Propiedad,2020-12-25,2020-12-29,2020-12-25,-34.919373,-58.020591,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Mensual,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Lote,Venta
3,dLHXKN5/sRZpm9Yk0yI2nA==,Propiedad,2020-12-25,2020-12-29,2020-12-25,-34.919455,-58.024807,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Mensual,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Lote,Venta
4,wtw/k887EPipd37UYHKb1Q==,Propiedad,2020-12-25,9999-12-31,2020-12-25,-34.364924,-58.783143,Argentina,Bs.As. G.B.A. Zona Norte,Escobar,...,NaN,18164.0,18164.0,NaN,NaN,Mensual,PANAMERICANA 47300,Nave principal 66 x 90 m: 6005 m2 cubiertos...,Otro,Venta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,SEeK1Z96Q3KuTKhNyREFDg==,Propiedad,2020-05-12,2020-09-02,2020-05-12,-34.900186,-56.176744,Uruguay,Montevideo,NaN,...,2.0,58.0,55.0,23000.0,UYU,Mensual,Apartamento en ALQUILER 2 dormitorios - Cordón:,Apartamento en ALQUILER 2 dormitorios - Cordón...,Departamento,Alquiler
999996,9rprv1XJCeNLnJP9dgWy0A==,Propiedad,2020-05-12,2020-05-28,2020-05-12,-34.886879,-56.188048,Uruguay,Montevideo,NaN,...,1.0,62.0,62.0,25000.0,UYU,Mensual,Apartamento alquiler a estrenar 2 dormitorios ...,Apartamento moderno a estrenar de 2 dormitorio...,Departamento,Alquiler
999997,ABzRp7AdRjDSFxKP1iNeYA==,Propiedad,2020-05-12,2020-08-11,2020-05-12,-34.891367,-56.098215,Uruguay,Montevideo,NaN,...,2.0,NaN,85.0,48000.0,UYU,Mensual,Casa - Malvín,"EN EL CORAZON DE MALVIN, CERCA DE TODO<br><br>...",Casa,Alquiler
999998,/dXMbrn3glBxu4WQBQu6IA==,Propiedad,2020-05-12,9999-12-31,2020-05-12,-34.462092,-57.842109,Uruguay,Colonia,NaN,...,1.0,135.0,135.0,35000.0,UYU,Mensual,House - Colonia del Sacramento,"Casa de tres dormitorios en muy linda cuadra, ...",Casa,Alquiler
